In [6]:
import feature_extractor as fe
import content_features as ctnfe
import url_features as urlfe
import tldextract
import external_features as trdfe
import re
import whois    

url, dom = fe.getPageContent("https://fast.com/")

# print(cf.nb_hyperlinks(dom))


ModuleNotFoundError: No module named 'pandas'

In [3]:
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup

def getPageContent(url):
    parsed = urlparse(url)
    url = parsed.scheme + '://'+ parsed.netloc
    try:
        page = requests.get(url)
    except:
        if not parsed.netloc.startswith('www'):
            url = parsed.scheme + '://www.'+ parsed.netloc
            page = requests.get(url)
    if page.status_code != 200:
        return None, None
    else:    
        return url, page.content

def getDOM(url):
    url, content = getPageContent(url)
    if content:
        soup = BeautifulSoup(content, 'html.parser')
        return soup
    else:
        return None

# Example usage
url = "https://www.wikipedia.org/"
dom = getDOM(url)
if dom:
    print(dom.prettify())
else:
    print("Failed to retrieve or parse the page.")



<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Wikipedia
  </title>
  <meta content="Wikipedia is a free online encyclopedia, created and edited by volunteers around the world and hosted by the Wikimedia Foundation." name="description"/>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)no-js(\s|$)/, "$1js-enabled$2" );
  </script>
  <meta content="initial-scale=1,user-scalable=yes" name="viewport"/>
  <link href="/static/apple-touch/wikipedia.png" rel="apple-touch-icon"/>
  <link href="/static/favicon/wikipedia.ico" rel="shortcut icon"/>
  <link href="//creativecommons.org/licenses/by-sa/4.0/" rel="license"/>
  <style>
   .sprite{background-image:linear-gradient(transparent,transparent),url(portal/wikipedia.org/assets/img/sprite-de847d1a.svg);background-repeat:no-repeat;display:inline-block;vertical-align:middle}.svg-Commons-logo_sister{background-position:0 0;width:47px;height:47px}.svg

In [5]:
# url=""
# fe.is_URL_accessible(url)
# len(fe.extract_features(url, True))
x = fe.extract_features(url, True)

In [6]:
x

['https://www.wikipedia.org/',
 26,
 17,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0.0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 3,
 3,
 3,
 0,
 9,
 9,
 0,
 6.0,
 6.0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 370,
 0.981081081081081,
 0.01891891891891892,
 0.0,
 0,
 0.0,
 0.2857142857142857,
 0.0,
 0.0,
 0,
 0,
 100.0,
 0,
 100.0,
 0.0,
 0,
 0,
 0,
 12.5,
 0,
 0,
 0,
 0,
 1,
 0,
 217,
 -1,
 0,
 0,
 1,
 -1]

In [7]:
len(x)

87

In [3]:
url = "https://en.wikipedia.org/wiki/Hugo_(software)"

In [4]:
def words_raw_extraction(domain, subdomain, path):
    w_domain = re.split(r"\-|\.|\/|\?|\=|\@|\&|\%|\:|\_", domain.lower())
    w_subdomain = re.split(r"\-|\.|\/|\?|\=|\@|\&|\%|\:|\_", subdomain.lower())   
    w_path = re.split(r"\-|\.|\/|\?|\=|\@|\&|\%|\:|\_", path.lower())
    raw_words = w_domain + w_path + w_subdomain
    w_host = w_domain + w_subdomain
    raw_words = list(filter(None, raw_words))
    return raw_words, list(filter(None, w_host)), list(filter(None, w_path))

In [5]:
Href = {'internals': [], 'externals': [], 'null': []}
Link = {'internals': [], 'externals': [], 'null': []}
Anchor = {'safe': [], 'unsafe': [], 'null': []}
Media = {'internals': [], 'externals': [], 'null': []}
Form = {'internals': [], 'externals': [], 'null': []}
CSS = {'internals': [], 'externals': [], 'null': []}
Favicon = {'internals': [], 'externals': [], 'null': []}
IFrame = {'visible': [], 'invisible': [], 'null': []}
Title = ''
Text = ''


In [6]:
import requests
from urllib.parse import urlparse
import concurrent.futures

class TimedOutExc(Exception):
    pass

def fetch_url(url):
    try:
        return requests.get(url, timeout=5)
    except requests.RequestException:
        return None

def is_URL_accessible(url):
    page = None
    parsed = urlparse(url)
    domain_url = f"{parsed.scheme}://{parsed.netloc}"
    www_domain_url = f"{parsed.scheme}://www.{parsed.netloc}"

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(fetch_url, url)
        try:
            page = future.result(timeout=5)
        except concurrent.futures.TimeoutError:
            page = None

        if not page or page.status_code != 200 or page.content in [b'', b' ']:
            future = executor.submit(fetch_url, domain_url)
            try:
                page = future.result(timeout=5)
            except concurrent.futures.TimeoutError:
                page = None

        if not page or page.status_code != 200 or page.content in [b'', b' ']:
            future = executor.submit(fetch_url, www_domain_url)
            try:
                page = future.result(timeout=5)
            except concurrent.futures.TimeoutError:
                page = None

    if page and page.status_code == 200 and page.content not in [b'', b' ']:
        return True, url, page
    else:
        return False, None, None

# Example usage
url = "https://www.wikipedia.org/"
state, iurl, page = is_URL_accessible(url)
if state:
    print("URL is accessible")
else:
    print("URL is not accessible")


URL is accessible


In [6]:


state, iurl, page = fe.is_URL_accessible(url)


: 

In [58]:
if state:
    content = page.content
    hostname, domain, path = fe.get_domain(url)
    extracted_domain = tldextract.extract(url)
    domain = extracted_domain.domain + '.' + extracted_domain.suffix
    subdomain = extracted_domain.subdomain
    tmp = url[url.find(extracted_domain.suffix):len(url)]
    pth = tmp.partition("/")
    path = pth[1] + pth[2]
    words_raw, words_raw_host, words_raw_path = words_raw_extraction(extracted_domain.domain, subdomain, pth[2])
    tld = extracted_domain.suffix
    parsed = urlparse(url)
    scheme = parsed.scheme
    
    Href, Link, Anchor, Media, Form, CSS, Favicon, IFrame, Title, Text = fe.extract_data_from_URL(
        hostname, content, domain, Href, Link, Anchor, Media, Form, CSS, Favicon, IFrame, Title, Text
    )

In [61]:
row = [url,
        # url-based features
        urlfe.url_length(url),
        urlfe.url_length(hostname),
        urlfe.having_ip_address(url),
        urlfe.count_dots(url),
        urlfe.count_hyphens(url),
        urlfe.count_at(url),
        urlfe.count_exclamation(url),
        urlfe.count_and(url),
        urlfe.count_or(url),
        urlfe.count_equal(url),
        urlfe.count_underscore(url),
        urlfe.count_tilde(url),
        urlfe.count_percentage(url),
        urlfe.count_slash(url),
        urlfe.count_star(url),
        urlfe.count_colon(url),
        urlfe.count_comma(url),
        urlfe.count_semicolumn(url),
        urlfe.count_dollar(url),
        urlfe.count_space(url),
        
        urlfe.check_www(words_raw),
        urlfe.check_com(words_raw),
        urlfe.count_double_slash(url),
        urlfe.count_http_token(path),
        urlfe.https_token(scheme),
        
        urlfe.ratio_digits(url),
        urlfe.ratio_digits(hostname),
        urlfe.punycode(url),
        urlfe.port(url),
        urlfe.tld_in_path(tld, path),
        urlfe.tld_in_subdomain(tld, subdomain),
        urlfe.abnormal_subdomain(url),
        urlfe.count_subdomain(url),
        urlfe.prefix_suffix(url),
    #    urlfe.random_domain(domain),
        urlfe.shortening_service(url),
        
        
        urlfe.path_extension(path),
        urlfe.count_redirection(page),
        urlfe.count_external_redirection(page, domain),
        urlfe.length_word_raw(words_raw),
        urlfe.char_repeat(words_raw),
        urlfe.shortest_word_length(words_raw),
        urlfe.shortest_word_length(words_raw_host),
        urlfe.shortest_word_length(words_raw_path),
        urlfe.longest_word_length(words_raw),
        urlfe.longest_word_length(words_raw_host),
        urlfe.longest_word_length(words_raw_path),
        urlfe.average_word_length(words_raw),
        urlfe.average_word_length(words_raw_host),
        urlfe.average_word_length(words_raw_path),
        
        urlfe.phish_hints(url),  
        urlfe.domain_in_brand(extracted_domain.domain),
        urlfe.brand_in_path(extracted_domain.domain,subdomain),
        urlfe.brand_in_path(extracted_domain.domain,path),
        urlfe.suspecious_tld(tld),
        urlfe.statistical_report(url, domain),

        
        # # # # content-based features
          ctnfe.nb_hyperlinks(Href, Link, Media, Form, CSS, Favicon),
          ctnfe.internal_hyperlinks(Href, Link, Media, Form, CSS, Favicon),
          ctnfe.external_hyperlinks(Href, Link, Media, Form, CSS, Favicon),
          ctnfe.null_hyperlinks(hostname, Href, Link, Media, Form, CSS, Favicon),
          ctnfe.external_css(CSS),
          ctnfe.internal_redirection(Href, Link, Media, Form, CSS, Favicon),
          ctnfe.external_redirection(Href, Link, Media, Form, CSS, Favicon),
          ctnfe.internal_errors(Href, Link, Media, Form, CSS, Favicon),
          ctnfe.external_errors(Href, Link, Media, Form, CSS, Favicon),
          ctnfe.login_form(Form),
          ctnfe.external_favicon(Favicon),
          ctnfe.links_in_tags(Link),
          ctnfe.submitting_to_email(Form),
          ctnfe.internal_media(Media),
          ctnfe.external_media(Media),
        #  # additional content-based features
          ctnfe.sfh(hostname,Form),
          ctnfe.iframe(IFrame),
          ctnfe.popup_window(Text),
          ctnfe.safe_anchor(Anchor),
          ctnfe.onmouseover(Text),
          ctnfe.right_clic(Text),
          ctnfe.empty_title(Title),
          ctnfe.domain_in_title(extracted_domain.domain, Title),
          ctnfe.domain_with_copyright(extracted_domain.domain, Text),
            
        # # # thirs-party-based features
          trdfe.whois_registered_domain(domain),
          trdfe.domain_registration_length(domain),
          # trdfe.domain_age(domain),
          trdfe.web_traffic(url),
          trdfe.dns_record(domain),
          trdfe.google_index(url),
        #     # trdfe.page_rank(key,domain),
        # # status
        # # status
        ]
print(row)

['https://www.wikipedia.org/', 26, 17, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 3, 3, 0, 9, 9, 0, 6.0, 6.0, 0, 0, 1, 0, 0, 0, 0, 370, 0.981081081081081, 0.01891891891891892, 0.0, 0, 0.0, 0.2857142857142857, 0.0, 0.0, 0, 0, 100.0, 0, 100.0, 0.0, 0, 0, 0, 12.5, 0, 0, 0, 0, 1, 1, -1, 0, 0, 1]


In [35]:
len(row)

85

In [9]:
import whois
import datetime

def get_domain_age(url):
  """
  This function attempts to fetch the estimated domain age of a website
  given its URL.

  Args:
      url (str): The URL of the website.

  Returns:
      str: The estimated domain age in a human-readable format (e.g., "10 years, 2 months").
          If the domain age cannot be determined, returns None.
  """
  try:
    # Extract the domain name from the URL (handle potential variations)
    try:
      import urllib.parse  # For Python 3
      parsed_url = urllib.parse.urlparse(url)
      domain = parsed_url.netloc
    except ImportError:
      from urlparse import urlparse  # For Python 2
      parsed_url = urlparse(url)
      domain = parsed_url.netloc

    # Use the whois library to get domain information
    domain_info = whois.whois(domain)

    # Look for creation date in WHOIS data (may vary based on registrar)
    for key in ['creation_date', 'created']:
      if key in domain_info:
        creation_date = domain_info[key]
        # Check if creation_date is a date object or a list
        if isinstance(creation_date, list):
          # Try to extract the first element if it's a list (assuming it's the date)
          try:
            creation_date = creation_date[0]
          except IndexError:
            # No usable date found in the list, continue iterating
            continue
        break
    else:
      # Creation date not found
      return None

    # Calculate the age difference in years and months (assuming creation_date is a date)
    today = datetime.date.today()
    age_years = today.year - creation_date.year
    age_months = today.month - creation_date.month


    # Handle negative months
    if age_months < 0:
      age_years -= 1
      age_months += 12

    days = age_years*365 + age_months*30

    # Return the age in a human-readable format
    return days
  except Exception as e:
    # WHOIS lookup failed
    # print(f"Error fetching domain age: {e}")
    return -1

# Example usage
url = "https://www.example.com"
domain_age = get_domain_age(url)

if domain_age:
  print(f"Estimated domain age for {url}: {domain_age}")
else:
  print(f"Could not determine domain age for {url}")


Estimated domain age for https://www.example.com: 10520


In [8]:
import whois
import datetime
from urllib.parse import urlparse

def get_domain_age(url):
    """
    This function attempts to fetch the estimated domain age of a website
    given its URL.

    Args:
        url (str): The URL of the website.

    Returns:
        int: The estimated domain age in days.
             If the domain age cannot be determined, returns -1.
    """
    try:
        # Extract the domain name from the URL
        parsed_url = urlparse(url)
        domain = parsed_url.netloc

        # Use the whois library to get domain information
        domain_info = whois.whois(domain)

        # Look for creation date in WHOIS data (may vary based on registrar)
        creation_date = None
        for key in ['creation_date', 'created']:
            if key in domain_info:
                creation_date = domain_info[key]
                # Check if creation_date is a date object or a list
                if isinstance(creation_date, list):
                    # Try to extract the first element if it's a list (assuming it's the date)
                    try:
                        creation_date = creation_date[0]
                    except IndexError:
                        continue
                break

        if creation_date is None:
            # Creation date not found
            return -1

        # Calculate the age difference in days
        today = datetime.date.today()
        domain_age_days = (today - creation_date.date()).days

        return domain_age_days
    except Exception as e:
        # WHOIS lookup failed
        # print(f"Error fetching domain age: {e}")
        return -1

# Example usage
url = "https://www.example.com"
domain_age = get_domain_age(url)

if domain_age != -1:
    print(f"Estimated domain age for {url}: {domain_age} days")
else:
    print(f"Could not determine domain age for {url}")


Estimated domain age for https://www.example.com: 10528 days
